<a href="https://colab.research.google.com/github/edwin-19/NLP-Framework-Comparison/blob/main/NLP_Nemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update && apt-get install -y libsndfile1 ffmpeg
!pip install Cython
!pip install nemo_toolkit[all]

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [85.2 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,734 kB]
Get:1

In [2]:
import nemo
import nemo.collections.nlp as nemo_nlp

[NeMo W 2022-05-11 07:04:39 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2022-05-11 07:04:39 experimental:28] Module <class 'nemo.collections.nlp.data.language_modeling.megatron.megatron_batch_samplers.MegatronPretrainingRandomBatchSampler'> is experimental, not ready for production and is not fully supported. Use at your own risk.
################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



In [15]:
import torch
from pprint import pprint
from omegaconf import OmegaConf
import numpy as np

In [6]:
zs_model = nemo_nlp.models.ZeroShotIntentModel.from_pretrained("zeroshotintent_en_bert_base_uncased")

[NeMo I 2022-05-11 07:05:42 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/zeroshotintent_en_bert_base_uncased/versions/1.4.1/files/zeroshotintent_en_bert_base_uncased.nemo to /root/.cache/torch/NeMo/NeMo_1.8.2/zeroshotintent_en_bert_base_uncased/bd0b8d430477bc6e329c79219ca77690/zeroshotintent_en_bert_base_uncased.nemo
[NeMo I 2022-05-11 07:06:04 common:747] Instantiating model from pre-trained checkpoint
[NeMo I 2022-05-11 07:06:09 tokenizer_utils:130] Getting HuggingFace AutoTokenizer with pretrained_model_name: bert-base-uncased, vocab_file: /tmp/tmp9zmf8uv8/9cf58d5590404f6ca6ffa00bbc4b1ac5_tokenizer.vocab_file, merges_files: None, special_tokens_dict: {}, and use_fast: False


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Using eos_token, but it is not set yet.
Using bos_token, but it is not set yet.
[NeMo W 2022-05-11 07:06:17 modelPT:216] You tried to register an artifact under config key=tokenizer.vocab_file but an artifact for it has already been registered.
[NeMo W 2022-05-11 07:06:18 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    file_name: train.tsv
    shuffle: true
    num_samples: -1
    batch_size: 32
    
[NeMo W 2022-05-11 07:06:18 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    file_name: dev_matched.tsv
    shuffle: false
    num_samples: -1
    batch_size: 32
    
[NeMo W 2022-05-11 07:06:18 nlp_overrides:224] Apex was not found. 

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[NeMo I 2022-05-11 07:06:49 save_restore_connector:209] Model ZeroShotIntentModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.8.2/zeroshotintent_en_bert_base_uncased/bd0b8d430477bc6e329c79219ca77690/zeroshotintent_en_bert_base_uncased.nemo.


In [11]:
queries = ['Fried Rice', 'Can you tell me if its sunny today', 'I want to eat rice', 'I am sad']
labels = ['food', 'weather', 'sentiment']
results = zs_model.predict(queries, labels, batch_size=10, multi_label=False)

for result in results:
  print('Sent: {}, Pred: {}, Score: {}'.format(result['sentence'], result['labels'][0], result['scores'][0]))

[NeMo I 2022-05-11 07:10:19 zero_shot_intent_dataset:234] Processing queries for inference
[NeMo I 2022-05-11 07:10:19 glue_benchmark_dataset:256] Writing example 0 of 12
[NeMo I 2022-05-11 07:10:19 glue_benchmark_dataset:338] *** Example ***
[NeMo I 2022-05-11 07:10:19 glue_benchmark_dataset:339] guid: query-0-label-0
[NeMo I 2022-05-11 07:10:19 glue_benchmark_dataset:340] tokens: [CLS] fried rice [SEP] this example is food . [SEP]
[NeMo I 2022-05-11 07:10:19 glue_benchmark_dataset:341] input_ids: 101 13017 5785 102 2023 2742 2003 2833 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
[NeMo I 2022-05-11 07:10:19 glue_benchmark_dataset:342] input_mask: 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [12]:
# Lets try multi labels at the same time
queries = ['I dont know what i am doing', 'Are we happy?', 'I was happy eariler this week but now no', 'Hello but not happy']
labels = ['sad', 'neutral', 'happy']
results = zs_model.predict(queries, labels, batch_size=10, multi_label=True)

for result in results:
  print('Sent: {}, Pred: {}, Score: {}'.format(result['sentence'], result['labels'], result['scores']))

[NeMo I 2022-05-11 07:12:33 zero_shot_intent_dataset:234] Processing queries for inference
[NeMo I 2022-05-11 07:12:33 glue_benchmark_dataset:256] Writing example 0 of 12
[NeMo I 2022-05-11 07:12:33 glue_benchmark_dataset:338] *** Example ***
[NeMo I 2022-05-11 07:12:33 glue_benchmark_dataset:339] guid: query-0-label-0
[NeMo I 2022-05-11 07:12:33 glue_benchmark_dataset:340] tokens: [CLS] i don ##t know what i am doing [SEP] this example is sad . [SEP]
[NeMo I 2022-05-11 07:12:33 glue_benchmark_dataset:341] input_ids: 101 1045 2123 2102 2113 2054 1045 2572 2725 102 2023 2742 2003 6517 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
[NeMo I 2022-05-11 07:12:33 glue_benchmark_dataset:342] input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

# Lets check out other models

In [22]:
model_list = nemo_nlp.modules.get_pretrained_lm_models_list()
tokenizers_list = nemo_nlp.modules.get_tokenizer_list()
pprint(model_list)
pprint(tokenizers_list)

['bert-base-uncased',
 'bert-large-uncased',
 'bert-base-cased',
 'bert-large-cased',
 'bert-base-multilingual-uncased',
 'bert-base-multilingual-cased',
 'bert-base-chinese',
 'bert-base-german-cased',
 'bert-large-uncased-whole-word-masking',
 'bert-large-cased-whole-word-masking',
 'bert-large-uncased-whole-word-masking-finetuned-squad',
 'bert-large-cased-whole-word-masking-finetuned-squad',
 'bert-base-cased-finetuned-mrpc',
 'bert-base-german-dbmdz-cased',
 'bert-base-german-dbmdz-uncased',
 'cl-tohoku/bert-base-japanese',
 'cl-tohoku/bert-base-japanese-whole-word-masking',
 'cl-tohoku/bert-base-japanese-char',
 'cl-tohoku/bert-base-japanese-char-whole-word-masking',
 'TurkuNLP/bert-base-finnish-cased-v1',
 'TurkuNLP/bert-base-finnish-uncased-v1',
 'wietsedv/bert-base-dutch-cased',
 'distilbert-base-uncased',
 'distilbert-base-uncased-distilled-squad',
 'distilbert-base-cased',
 'distilbert-base-cased-distilled-squad',
 'distilbert-base-german-cased',
 'distilbert-base-multilingu

In [29]:
# use any pretrained model name from the list above
pretrained_model_name = 'distilbert-base-multilingual-cased'
config = {"language_model": {"pretrained_model_name": pretrained_model_name}, "tokenizer": {'tokenizer_name':pretrained_model_name}}
omega_conf = OmegaConf.create(config)
lang_model = nemo_nlp.modules.get_lm_model(cfg=omega_conf)
tokenizer = nemo_nlp.modules.get_tokenizer(**config['tokenizer'])

Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertEncoder: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'voca

[NeMo I 2022-05-11 07:23:21 tokenizer_utils:130] Getting HuggingFace AutoTokenizer with pretrained_model_name: distilbert-base-multilingual-cased, vocab_file: None, merges_files: None, special_tokens_dict: {}, and use_fast: False


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Using eos_token, but it is not set yet.
Using bos_token, but it is not set yet.


In [25]:
pprint(omega_conf)

{'language_model': {'pretrained_model_name': 'distilbert-base-multilingual-cased'}, 'tokenizer': {'tokenizer_name': 'distilbert-base-multilingual-cased'}}


{'input_ids': tensor([[  101, 31178, 10315,   102]]), 'attention_mask': tensor([[1, 1, 1, 1]])}

In [36]:
# Lets see if it runs like a normal hf model
input_ids = tokenizer.tokenizer('Hello World', return_tensors='pt')
outputs = lang_model(**input_ids)

In [37]:
# Hmm seems like it needs to be adjusted a bit according
# Nemo uses its own wrapper around the tokenizer class
outputs.shape

torch.Size([1, 4, 768])